In [5]:
from flask import Flask, jsonify, request, render_template_string
from pyngrok import ngrok
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
import nltk

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

app = Flask(__name__)

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# HTML and CSS content
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Sentiment Analysis</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            background-color: #f4f4f4;
            margin: 0;
            padding: 0;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
        }
        .container {
            background-color: white;
            padding: 20px;
            border-radius: 8px;
            box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.1);
        }
        h1 {
            text-align: center;
        }
        textarea {
            width: 100%;
            height: 100px;
            margin-bottom: 20px;
            padding: 10px;
            font-size: 16px;
        }
        button {
            display: block;
            width: 100%;
            padding: 10px;
            background-color: #007BFF;
            color: white;
            border: none;
            border-radius: 5px;
            font-size: 16px;
            cursor: pointer;
            margin-bottom: 10px;
        }
        button:hover {
            background-color: #0056b3;
        }
        #result {
            margin-top: 20px;
        }
        #sentiment, #comment {
            font-weight: bold;
        }
        .close-button {
            background-color: red;
            margin-top: 10px;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Sentiment Analysis</h1>
        <form id="reviewForm">
            <textarea id="reviewText" placeholder="Enter your review"></textarea>
            <button type="button" onclick="analyzeReview()">Analyze</button>
            <button type="button" class="close-button" onclick="closeWindow()">Close</button>
        </form>
        <div id="result">
            <h2>Sentiment: <span id="sentiment"></span></h2>
            <h3>Comment: <span id="comment"></span></h3>
        </div>
    </div>

    <script>
        async function analyzeReview() {
            const reviewText = document.getElementById('reviewText').value;
            const response = await fetch('/analyze', {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify({ text: reviewText })
            });
            const data = await response.json();
            document.getElementById('sentiment').textContent = data.sentiment;
            document.getElementById('comment').textContent = data.comment;
        }

        function closeWindow() {
            window.close();
        }
    </script>
</body>
</html>
"""

def preprocess_text(text):
    tokens = word_tokenize(text)
    filtered_tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens if word.lower() not in stop_words and word.isalpha()]
    return ' '.join(filtered_tokens)

def analyze_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

def generate_comment(polarity):
    if polarity > 0.5:
        return "Great product! I love it."
    elif polarity < -0.5:
        return "Disappointing product. I wouldn't recommend it."
    elif polarity >= -0.5 and polarity <= 0.5:
        return "Neutral product. It's average."
    else:
        return "Average product. It's okay."

@app.route('/')
def index():
    return render_template_string(html_template)

@app.route('/analyze', methods=['POST'])
def analyze():
    data = request.get_json()
    text = data.get('text', '')

    filtered_text = preprocess_text(text)
    sentiment = analyze_sentiment(filtered_text)
    comment = generate_comment(sentiment)

    return jsonify({'sentiment': sentiment, 'comment': comment})

# Add your ngrok authtoken here
ngrok.set_auth_token("2mby1BZP6kg9hwPEkHkiA530iHv_2PhdU9JNP4p5bU2N93rMQ")

# Start ngrok tunnel
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

# Start Flask app
app.run(port=5000)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Public URL: NgrokTunnel: "https://2073-34-81-53-69.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [22/Sep/2024 17:26:42] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Sep/2024 17:26:42] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Sep/2024 17:26:51] "POST /analyze HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Sep/2024 17:27:09] "POST /analyze HTTP/1.1" 200 -


In [1]:
!pip install Flask pyngrok

